# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse    Shipments  \
0  Apr 25 2023  9:55AM  261739        10  Enova-11710   
1  Apr 25 2023  9:50AM  261738        15   PNC442915A   
2  Apr 25 2023  9:50AM  261738        15   PNC446496A   
3  Apr 25 2023  9:50AM  261738        15   PNC446513B   
4  Apr 25 2023  9:50AM  261738        15   PNC446514A   
5  Apr 25 2023  9:49AM  261737        10      9416819   
6  Apr 25 2023  9:47AM  261736        15      9416247   
7  Apr 25 2023  9:47AM  261736        15      9416248   
8  Apr 25 2023  9:47AM  261736        15      9416249   
9  Apr 25 2023  9:47AM  261736        15      9416250   

                          Customer ShipmentStatus  
0                      Emerginnova       Released  
1             Person & Covey, Inc.       Released  
2             Person & Covey, Inc.       Released  
3             Person & Covey, Inc.       Released  
4             Person & Covey, Inc.       Released  
5                   Citieffe, Inc.       Released  
6  Brookfield Pharmaceuticals, LLC       Released  
7  Brookfield Pharmaceuticals, LLC       Released  
8  Brookfield Pharmaceuticals, LLC       Released  
9  Brookfield Pharmaceuticals, LLC       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
17  261735       Released          2
18  261736       Released         22
19  261737       Released          1
20  261738       Released          4
21  261739       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Executing  Released
id                                 
261735                2.0       2.0
261736                NaN      22.0
261737                NaN       1.0
261738                NaN       4.0
261739                NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Executing  Released
id                                 
261641                1.0       0.0
261643                0.0      22.0
261649                1.0       0.0
261652                0.0       1.0
261692                0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus  Executing  Released
id                                 
261641                  1         0
261643                  0        22
261649                  1         0
261652                  0         1
261692                  0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Executing  Released
0               261641          1         0
1               261643          0        22
2               261649          1         0
3               261652          0         1
4               261692          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus      id Executing Released
0               261641         1         
1               261643                 22
2               261649         1         
3               261652                  1
4               261692                  1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                         Customer
0   Apr 25 2023  9:55AM  261739        10                      Emerginnova
1   Apr 25 2023  9:50AM  261738        15             Person & Covey, Inc.
5   Apr 25 2023  9:49AM  261737        10                   Citieffe, Inc.
6   Apr 25 2023  9:47AM  261736        15  Brookfield Pharmaceuticals, LLC
28  Apr 25 2023  9:43AM  261643        15            Alliance Pharma, Inc.
50  Apr 25 2023  9:42AM  261704        15            Alliance Pharma, Inc.
53  Apr 25 2023  9:41AM  261735        10                 Eywa Pharma Inc.
57  Apr 25 2023  9:30AM  261734        19             Emersa Waterbox, LLC
61  Apr 25 2023  9:10AM  261729        10              Digital Brands, LLC
64  Apr 25 2023  8:55AM  261726        12                 SD Head USA, LLC

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                         Customer  \
0  Apr 25 2023  9:55AM  261739        10                      Emerginnova   
1  Apr 25 2023  9:50AM  261738        15             Person & Covey, Inc.   
2  Apr 25 2023  9:49AM  261737        10                   Citieffe, Inc.   
3  Apr 25 2023  9:47AM  261736        15  Brookfield Pharmaceuticals, LLC   
4  Apr 25 2023  9:43AM  261643        15            Alliance Pharma, Inc.   
5  Apr 25 2023  9:42AM  261704        15            Alliance Pharma, Inc.   
6  Apr 25 2023  9:41AM  261735        10                 Eywa Pharma Inc.   
7  Apr 25 2023  9:30AM  261734        19             Emersa Waterbox, LLC   
8  Apr 25 2023  9:10AM  261729        10              Digital Brands, LLC   
9  Apr 25 2023  8:55AM  261726        12                 SD Head USA, LLC   

  Executing Released  
0                  1  
1                  4  
2                  1  
3                 22  
4                 22  
5                  3  
6         2        2  
7                  4  
8         1        2  
9         1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                         Customer  \
0  Apr 25 2023  9:55AM  261739        10                      Emerginnova   
1  Apr 25 2023  9:50AM  261738        15             Person & Covey, Inc.   
2  Apr 25 2023  9:49AM  261737        10                   Citieffe, Inc.   
3  Apr 25 2023  9:47AM  261736        15  Brookfield Pharmaceuticals, LLC   
4  Apr 25 2023  9:43AM  261643        15            Alliance Pharma, Inc.   
5  Apr 25 2023  9:42AM  261704        15            Alliance Pharma, Inc.   
6  Apr 25 2023  9:41AM  261735        10                 Eywa Pharma Inc.   
7  Apr 25 2023  9:30AM  261734        19             Emersa Waterbox, LLC   
8  Apr 25 2023  9:10AM  261729        10              Digital Brands, LLC   
9  Apr 25 2023  8:55AM  261726        12                 SD Head USA, LLC   

  Released Executing  
0        1            
1        4            
2        1            
3       22            
4       22            
5        3            
6        2         2  
7        4            
8        2         1  
9                  1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                         Customer  \
0  Apr 25 2023  9:55AM  261739        10                      Emerginnova   
1  Apr 25 2023  9:50AM  261738        15             Person & Covey, Inc.   
2  Apr 25 2023  9:49AM  261737        10                   Citieffe, Inc.   
3  Apr 25 2023  9:47AM  261736        15  Brookfield Pharmaceuticals, LLC   
4  Apr 25 2023  9:43AM  261643        15            Alliance Pharma, Inc.   

  Released Executing  
0        1            
1        4            
2        1            
3       22            
4       22

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


Date      id Warehouse                         Customer  \
0  Apr 25 2023  9:55AM  261739        10                      Emerginnova   
1  Apr 25 2023  9:50AM  261738        15             Person & Covey, Inc.   
2  Apr 25 2023  9:49AM  261737        10                   Citieffe, Inc.   
3  Apr 25 2023  9:47AM  261736        15  Brookfield Pharmaceuticals, LLC   
4  Apr 25 2023  9:43AM  261643        15            Alliance Pharma, Inc.   

   Released  Executing  
0       1.0        NaN  
1       4.0        NaN  
2       1.0        NaN  
3      22.0        NaN  
4      22.0        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                         Customer  \
0  Apr 25 2023  9:55AM  261739        10                      Emerginnova   
1  Apr 25 2023  9:50AM  261738        15             Person & Covey, Inc.   
2  Apr 25 2023  9:49AM  261737        10                   Citieffe, Inc.   
3  Apr 25 2023  9:47AM  261736        15  Brookfield Pharmaceuticals, LLC   
4  Apr 25 2023  9:43AM  261643        15            Alliance Pharma, Inc.   

   Released  Executing  
0       1.0        0.0  
1       4.0        0.0  
2       1.0        0.0  
3      22.0        0.0  
4      22.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing
Warehouse                              
10         1308581       6.0        4.0
12          261726       0.0        1.0
15         1046821      51.0        0.0
19         1308572      26.0        7.0
20          523301       1.0        1.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing
0        10  1308581       6.0        4.0
1        12   261726       0.0        1.0
2        15  1046821      51.0        0.0
3        19  1308572      26.0        7.0
4        20   523301       1.0        1.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing
0        10       6.0        4.0
1        12       0.0        1.0
2        15      51.0        0.0
3        19      26.0        7.0
4        20       1.0        1.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released    6.0
1        12  Released    0.0
2        15  Released   51.0
3        19  Released   26.0
4        20  Released    1.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse   10   12    15    19   20
Status                              
Executing  4.0  1.0   0.0   7.0  1.0
Released   6.0  0.0  51.0  26.0  1.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status   10   12    15    19   20
0          Executing  4.0  1.0   0.0   7.0  1.0
1           Released  6.0  0.0  51.0  26.0  1.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status   10   12    15    19   20
0  Executing  4.0  1.0   0.0   7.0  1.0
1   Released  6.0  0.0  51.0  26.0  1.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()